In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys

dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)

os.chdir('..')

In [2]:
from utils import metrics, core_models
from utils.model_utils import fix_seeds
from utils.datasets import CPDDatasets
from utils.cpd_models import CPDModel

import yaml

from torch_sgld import SGLD, CosineLR

import torch
import torch.nn as nn
from torch.utils.data import Dataset

import pytorch_lightning as pl

from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

%load_ext autoreload
%autoreload
%matplotlib inline

comet_ml is installed but `COMET_API_KEY` is not set.


In [3]:
import math
from torch.optim.lr_scheduler import _LRScheduler


class ABAnnealingLR(_LRScheduler):
  """Step size scheduler for SGLD.

  a and b are computed based on start and final step size.

  .. math::
    \epsilon_t = a(b + t)^{-\gamma}

  .. _SGLD\: Bayesian Learning via Stochastic Gradient Langevin Dynamics:
        https://icml.cc/2011/papers/398_icmlpaper.pdf
  """
  def __init__(self, optimizer, final_lr, gamma, T_max, last_epoch=-1, verbose=False):
    self.final_lr = final_lr
    self.gamma = gamma
    self.T_max = T_max

    super().__init__(optimizer, last_epoch, verbose)

  def get_lr(self):
    if not self._get_lr_called_within_step:
      warnings.warn("To get the last learning rate computed by the scheduler, "
                    "please use `get_last_lr()`.", UserWarning)

    if self.last_epoch == 0:
      return self.base_lrs

    new_lrs = []
    for base_lr, group in zip(self.base_lrs, self.optimizer.param_groups):
      if self.last_epoch > self.T_max:
        new_lrs.append(group['lr'])
      else:
        b = self.T_max / ((base_lr / self.final_lr) * math.exp(1 / self.gamma) - 1.)
        a = base_lr * b ** self.gamma

        new_lr = a / (b + self.last_epoch) ** self.gamma
        new_lrs.append(new_lr)

    return new_lrs


In [4]:
class CPDModelSGLD(CPDModel):
    """Pytorch Lightning wrapper for change point detection models."""
    def __init__(
        self,
        loss_type: str,
        args: dict,
        model: nn.Module,
        train_dataset: Dataset,
        test_dataset: Dataset,
    ) -> None:
        """Initialize CPD model.

        :param experiment_type: type of data used for training
        :param loss_type: type of loss function for training special CPD or common BCE loss
        :param args: dict with supplementary argumemnts
        :param model: base model
        :param train_dataset: train data
        :param test_dataset: test data
        """
        super().__init__(loss_type, args, model, train_dataset, test_dataset)
        
        self.momentum = args["learning"]["momentum"]
        self.temperature = args["learning"]["temperature"] 
        
        #self.n_cycles = args["learning"]["n_cycles"]
        #self.n_samples = args["learning"]["n_samples"]

        self.T_max = args["learning"]["T_max"]
        
        self.final_lr =  args["learning"]["final_lr"]
        self.gamma = args["learning"]["gamma"]
        self.last_epoch = args["learning"]["last_epoch"] 

    def configure_optimizers(self):
        """Initialize optimizer.

        :return: optimizer for training CPD model
        """
        optimizer = SGLD(
            self.model.parameters(),
            lr=self.learning_rate,
            temperature=self.temperature,
            momentum=self.momentum
        )
        
        #scheduler = CosineLR(
        #    optimizer, n_cycles=self.n_cycles, n_samples=self.n_samples, T_max=self.T_max
        #)
        
        scheduler = ABAnnealingLR(
            optimizer,
            #initial_lr=self.initial_lr,
            final_lr=self.final_lr,
            gamma=self.gamma,
            T_max=self.T_max,
            #last_epoch=-self.last_epoch,
            verbose=False 
        )
         
        return [optimizer], [scheduler]

In [5]:
SEED = 42
fix_seeds(SEED)

In [6]:
model_type = "seq2seq"

experiments_name = "human_activity"

path_to_config = "configs/" + experiments_name + "_" + model_type + ".yaml"

with open(path_to_config, 'r') as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type

args_config["loss_type"] = "bce"
args_config["num_workers"] = 2
args_config["learning"]["gpus"] = 1

args_config["learning"]["epochs"] = 100

In [8]:
train_dataset, test_dataset = CPDDatasets(experiments_name).get_dataset_()
train_dataset[0][0].shape

(20, 28)

In [10]:
args_config["learning"]["lr"] = 1e-3
args_config["learning"]["momentum"] = 0.9
args_config["learning"]["temperature"] = 5

# for CosineLR
#args_config["learning"]["n_cycles"] = 4
#args_config["learning"]["n_samples"] = 2000
#args_config["learning"]["T_max"] = 10000

# for ABAnnealingLR
args_config["learning"]["final_lr"] = 1e-4
args_config["learning"]["gamma"] = 1
args_config["learning"]["T_max"] = 10000
args_config["learning"]["last_epoch"] = -1

core_model = core_models.BaseRnn(
    input_size=args_config["model"]["input_size"],
    hidden_dim=args_config["model"]["hidden_dim"],
    n_layers=args_config["model"]["n_layers"],
    drop_prob=args_config["model"]["drop_prob"],
    layer_norm=args_config["model"]["layer_norm"]
)
core_model

bce_model = CPDModelSGLD(
    loss_type="bce",
    args=args_config,
    model=core_model,
    train_dataset=train_dataset,
    test_dataset=test_dataset
)

logger = CometLogger(
    save_dir='logs/bce/sgld',
    api_key='agnHNC2vEt7tOxnnxT4LzYf7Y',
    project_name='cpd-with-uncertainty',
    workspace='stalex2902',
    experiment_name='BCE-SGLD_ABAnnealing_HAR_default_params',
    display_summary_level=0
)

trainer = pl.Trainer(
    max_epochs=100,
    gpus=1,
    benchmark=True,
    check_val_every_n_epoch=1,
    gradient_clip_val=0.,
    logger=logger,
    # use early stopping
    #callbacks=EarlyStopping(monitor="val_loss", min_delta=0, patience=10)
)

trainer.fit(bce_model)

CometLogger will be initialized in online mode
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type    | Params
----------------------------------
0 | model | BaseRnn | 1.2 K 
1 | loss  | BCELoss | 0     
----------------------------------
1.2 K     Trainable params
0         Non-trainable params
1.2 K     Total params
0.005     Total estimated model params size (MB)


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/stalex2902/cpd-with-uncertainty/f26e37efc6264947ab5b220c6f50567b



COMET WARNING: Failed to log system metrics: [sys.ram,sys.cpu,sys.load]


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.


In [11]:
core_model_adam = core_models.BaseRnn(
    input_size=args_config["model"]["input_size"],
    hidden_dim=args_config["model"]["hidden_dim"],
    n_layers=args_config["model"]["n_layers"],
    drop_prob=args_config["model"]["drop_prob"],
    layer_norm=args_config["model"]["layer_norm"]
)

bce_model_adam = CPDModel(
    loss_type="bce",
    args=args_config,
    model=core_model_adam,
    train_dataset=train_dataset,
    test_dataset=test_dataset
)
bce_model_adam

CPDModel(
  (model): BaseRnn(
    (lstm): LSTM(28, 8, batch_first=True, dropout=0.5)
    (linear): Linear(in_features=8, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (loss): BCELoss()
)

In [12]:
logger = CometLogger(
    save_dir='logs/bce/sgld',
    api_key='agnHNC2vEt7tOxnnxT4LzYf7Y',
    project_name='cpd-with-uncertainty',
    workspace='stalex2902',
    experiment_name='BCE-Adam HAR',
    display_summary_level=0
)

trainer = pl.Trainer(
    max_epochs=100,
    gpus=1,
    benchmark=True,
    check_val_every_n_epoch=1,
    gradient_clip_val=0.,
    logger=logger,
    # use early stopping
    #callbacks=EarlyStopping(monitor="val_loss", min_delta=0, patience=10)
)

trainer.fit(bce_model_adam)

CometLogger will be initialized in online mode
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type    | Params
----------------------------------
0 | model | BaseRnn | 1.2 K 
1 | loss  | BCELoss | 0     
----------------------------------
1.2 K     Trainable params
0         Non-trainable params
1.2 K     Total params
0.005     Total estimated model params size (MB)


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/stalex2902/cpd-with-uncertainty/9453be7f259e4a0c96d948288668ce77



COMET WARNING: Failed to log system metrics: [sys.ram,sys.cpu,sys.load]


COMET WARNING: Failed to log system metrics: [sys.ram,sys.cpu,sys.load]


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET INFO: Please wait for metadata to finish uploading (timeout is 3600 seconds)
COMET INFO: Uploading 1 metrics, params and output messages
